In [1]:
import pandas as pd
import pickle

# Load processed data
train_df = pd.read_csv("../data/processed/train_ratings.csv")
movies = pd.read_csv("../data/raw/movies.csv")

# Load trained SVD model
with open("../models/svd_model.pkl", "rb") as f:
    svd = pickle.load(f)

print("Model and data loaded ")


Model and data loaded 


In [2]:
def get_unseen_movies(user_id, train_df, movies_df):
    seen_movies = train_df[train_df["userId"] == user_id]["movieId"].unique()
    all_movies = movies_df["movieId"].unique()
    
    unseen_movies = list(set(all_movies) - set(seen_movies))
    return unseen_movies


In [3]:
def recommend_top_k(user_id, k=10):
    unseen_movies = get_unseen_movies(user_id, train_df, movies)
    
    predictions = []
    for movie_id in unseen_movies:
        pred = svd.predict(user_id, movie_id)
        predictions.append((movie_id, pred.est))
    
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_k = predictions[:k]
    
    return top_k


In [4]:
user_id = 10  # you can change this
top_10 = recommend_top_k(user_id, k=10)

top_10


[(1233, 4.540695815289157),
 (142488, 4.230676123697445),
 (1387, 4.206837255378837),
 (3683, 4.182469094299172),
 (162, 4.165865416411786),
 (106100, 4.100412794385467),
 (750, 4.034204714817211),
 (475, 4.030701171757622),
 (1250, 4.002749808782517),
 (3703, 3.9800463542336137)]

In [5]:
def show_recommendations(top_k, movies_df):
    result = []
    for movie_id, score in top_k:
        title = movies_df[movies_df["movieId"] == movie_id]["title"].values[0]
        result.append((title, round(score, 3)))
    return result

show_recommendations(top_10, movies)


[('Boot, Das (Boat, The) (1981)', 4.541),
 ('Spotlight (2015)', 4.231),
 ('Jaws (1975)', 4.207),
 ('Blood Simple (1984)', 4.182),
 ('Crumb (1994)', 4.166),
 ('Dallas Buyers Club (2013)', 4.1),
 ('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
  4.034),
 ('In the Name of the Father (1993)', 4.031),
 ('Bridge on the River Kwai, The (1957)', 4.003),
 ('Road Warrior, The (Mad Max 2) (1981)', 3.98)]

Top-K Recommendation:
For a given user, unseen movies were ranked based on predicted ratings from the SVD model, and the top-10 highest scoring items were recommended.